# Importing the necessary libraries
The absolute first thing we must do is to import all the necessary libraries for this project. 

In [159]:
# Imports
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
device = ("cuda" if torch.cuda.is_available() else ("mps" if torch.has_mps else "cpu"))
print(f"device: {device}")

device: mps


# Reading data
After importing everything we need, we have to read the data from the files. We also have to pre-process the data slightly, by e.g. resizing the images. 

In [160]:
# Reading the data from the folders as well as creating dataloaders
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize(size=(224, 224))])
training_data = torchvision.datasets.ImageFolder("data/train/", transform=transform)
test_data = torchvision.datasets.ImageFolder("data/test/", transform=transform)

batch_size = 256
training_loader = DataLoader(dataset=training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size)

print(f"Number of training examples: {len(training_data)}")
print(f"Number of test examples: {len(test_data)}")

Number of training examples: 15557
Number of test examples: 4002


# Visualizing some of the data
This code visualizes some of the data in a grid, so we know what we are dealing with. 

In [161]:
classes = os.listdir("./data/train")
print(f"classes: {classes}")

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(training_loader)
images, labels = next(dataiter)

# show images
# imshow(torchvision.utils.make_grid(images))
# print labels
# print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

classes: ['Urticaria Hives', 'Seborrheic Keratoses and other Benign Tumors', 'Poison Ivy Photos and other Contact Dermatitis', 'Acne and Rosacea Photos', '.DS_Store', 'Vascular Tumors', 'Eczema Photos', 'Psoriasis pictures Lichen Planus and related diseases', 'Exanthems and Drug Eruptions', 'Lupus and other Connective Tissue diseases', 'Scabies Lyme Disease and other Infestations and Bites', 'Bullous Disease Photos', 'Nail Fungus and other Nail Disease', 'Tinea Ringworm Candidiasis and other Fungal Infections', 'Systemic Disease', 'Light Diseases and Disorders of Pigmentation', 'Atopic Dermatitis Photos', 'Warts Molluscum and other Viral Infections', 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions', 'Melanoma Skin Cancer Nevi and Moles', 'Vasculitis Photos', 'Cellulitis Impetigo and other Bacterial Infections', 'Hair Loss Photos Alopecia and other Hair Diseases', 'Herpes HPV and other STDs Photos']


/opt/homebrew/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


# Creating a CNN classifier
The following code creates a CNN that is used in order to classify the images. Then, we are going to train this classifier, and finally test the classifier on the test set. 

In [162]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_size = (224, 224)
        self.num_channels = 16
        self.num_labels = 23

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3, padding=1)

        self.fc1 = nn.Linear(
            (self.input_size[0] // 4) * (self.input_size[1] // 4) * self.num_channels,
            120,
        )
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, self.num_labels)

    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = self.pool(x)

        x = torch.flatten(x, 1)  # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [163]:
net = Net()
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

## Training the neural network

In [165]:
from time import time

num_epochs = 50

start = time()
epoch_losses = []
for epoch in range(num_epochs):  # loop over the dataset multiple times
    epoch_loss = 0
    for i, data in enumerate(training_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        epoch_loss += loss.item()

        loss.backward()
        optimizer.step()

    # Gathering statistics 
    end = time()
    epoch_losses.append(epoch_loss)
    print(f"Epoch: {epoch + 1}, loss: {epoch_loss:.3f}, time: {end - start:.2f}s")

end = time()
total_time = end - start
print(f'Finished Training, took {total_time:.2f} seconds.')

/opt/homebrew/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


Epoch: 1, loss: 168.785, time: 77.57s
Epoch: 2, loss: 163.367, time: 163.22s
Epoch: 3, loss: 158.658, time: 241.75s
Epoch: 4, loss: 154.696, time: 322.24s
Epoch: 5, loss: 151.497, time: 400.47s
Epoch: 6, loss: 147.543, time: 477.48s
Epoch: 7, loss: 142.911, time: 560.26s
Epoch: 8, loss: 138.153, time: 640.93s
Epoch: 9, loss: 133.800, time: 717.70s
Epoch: 10, loss: 128.108, time: 796.75s
Epoch: 11, loss: 121.238, time: 873.80s
Epoch: 12, loss: 114.030, time: 954.52s
Epoch: 13, loss: 104.592, time: 1035.57s
Epoch: 14, loss: 94.438, time: 1112.20s
Epoch: 15, loss: 82.399, time: 1192.06s
Epoch: 16, loss: 70.600, time: 1270.83s
Epoch: 17, loss: 58.559, time: 1349.72s
Epoch: 18, loss: 48.428, time: 1427.15s
Epoch: 19, loss: 38.200, time: 1506.24s
Epoch: 20, loss: 29.778, time: 1582.73s
Epoch: 21, loss: 23.914, time: 1659.30s
Epoch: 22, loss: 19.371, time: 1736.85s
Epoch: 23, loss: 15.538, time: 1813.68s
Epoch: 24, loss: 12.484, time: 1890.22s
Epoch: 25, loss: 10.800, time: 1969.53s
Epoch: 26

## Evaluating the neural network

In [166]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

Accuracy of the network on the test images: 29 %
